<a href="https://colab.research.google.com/github/rchicangana/AnalisisDatosProyecto/blob/IVAN/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESTADO DEL ARTE

- La imposibilidad de almacenar la energía eléctrica hace de la predicción una herramienta indispensable a la hora de gestionar eficientemente su producción. La modelización de la
demanda de energía eléctrica permite al ente productor contar con un conocimiento más refinado de los mercados y de los usuarios del sistema, así como con una mejor posición mediante la reducción de la incertidumbre en la toma de decisiones.  

- Se consultaron las Bases de Datos SciELO (Scientific Electronic Library Online) y RedAlyc (Red de Revistas Científicas de América Latina y el Caribe, España y Portugal), EBSCO, Springer Link, IEEE Xplorer y Google Académico.

- Para medir la relevancia y pertinencia de los documentos se tomaron como indicadores: que los artículos presentaran las técnicas utilizadas basadas en Inteligencia Artificial, así como metodologías y estudios de casos concretos dentro de las áreas que abarca la ingeniería eléctrica.

- Destacan dentro de las áreas de impacto la predicción de la producción y consumo eléctrico, lo cual se refiere a estudios realizados para establecer valores futuros del consumo eléctrico de una edificación particular, un complejo habitacional, industrial o incluso una ciudad o país completos. Como se indicó previamente, la adecuada predicción de estos datos es de gran importancia para planificar las fuentes de energía eléctrica y los costos asociados para los consumidores.

- La inteligencia artificial ofrece en la actualidad una diversidad de técnicas para la predicción de variables. Es la presencia de estas técnicas, en la forma de algoritmos implementados, que permiten relacionar las publicaciones con la inteligencia artificial. La diversidad de opciones de algoritmos generados se refleja en los artículos analizados
Las redes neuronales artificiales y las máquinas de vectores de soporte son las predominantes entre todas las utilizadas. Es importante destacar que es una práctica común la utilización de más de una técnica en un estudio de predicción, para realizar comparaciones y proponer la adopción de aquella técnica que haya dado mejores resultados. tambien se comentan tecnicas como Gradient Boosting y Modelos de Markov.

Entre las variables utilizadas para los analisis se encuentran principalmente la Temperatura, Humedad y la fecha de medicion


-Los resultados muestran una tendencia creciente en el interés de aprovechar las posibilidades de la inteligencia artificial a la tarea de predicción de variables, y confirman la mayor representatividad de los casos en la predicción de producción y consumo energético, y en variables relacionadas con energías renovables, tales como las condiciones del viento y de energía solar. La predominancia del idioma de las publicaciones es el inglés, y en países como China, Estados Unidos y Colombia es donde se ha dado la mayor cantidad de estudios.

-Los artículos más citados en la temática tratan sobre predicción de consumo eléctrico en edificaciones particulares, y de predicción de energía solar. Las predicciones son de utilidad para labores de planificación, estimación de costos y previsión de escenarios de inversión y aprovechamiento de recursos.

-El análisis realizado en el estado del arte deja patente la actualidad y necesidad del tema de inteligencia artificial para la ingeniería eléctrica, especialmente el conocimiento de las técnicas más utilizadas para los sistemas de predicción determinadas en el presente estudio, como lo son las redes neuronales artificiales, las máquinas de vectores de soporte y los árboles de decisión. Por otra parte, el creciente interés que muestran las referencias puede interpretarse como un área de oportunidad para aprovechar las ventajas de los estudios de predicción de variables con los nuevos métodos de inteligencia artificial en lugares donde aún no se han implementado.


## Articulos consultados

- https://www.sciencedirect.com/science/article/pii/S2352484724007315
- https://ciencialatina.org/index.php/cienciala/article/view/2502
- https://www.revistaespacios.com/a17v38n22/a17v38n21p03.pdf
- http://www.scielo.org.co/scielo.php?script=sci_arttext&pid=S1794-12372016000200009
- https://pmc.ncbi.nlm.nih.gov/articles/PMC10948898/
- https://www.redalyc.org/pdf/462/46247652001.pdf
- https://www.kaggle.com/datasets/fedesoriano/electric-power-consumption

### Los articulos mas citados en la publicaciones fueron


| Autores                                                       | Título del   artículo                                                                                                                                                                                        | Nombre de   la revista/publicación                           | Año  | Cantidad de citas |
|---------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------|------|-------------------|
| Gonzalez,   P. A., y Zamarreno, J. M..                        | Prediction of hourly energy   consumption in buildings based on a feedback artificial neural network                                                                                                         | Energy and buildings                                         | 2005 | 392               |
| Jain,   R. K., Smith, K. M., Culligan, P. J., y Taylor, J. E. | Forecasting energy consumption   of multi-family residential buildings using support vector regression:   Investigating the impact of temporal and spatial monitoring granularity on   performance accuracy. | Applied Energy                                               | 2014 | 379               |
| Sharma,   N., Sharma, P., Irwin, D., y Shenoy, P.             | Predicting solar generation from   weather forecasts using machine learning.                                                                                                                                 | IEEE international conference on   smart grid communications | 2011 | 370               |
| Yang,   J., Rivard, H., y Zmeureanu, R.                       | On-line building energy   prediction using adaptive artificial neural networks.                                                                                                                              | Energy and buildings,                                        | 2005 | 368               |
| Ekici,   B. B., y Aksoy, U. T.                                | Prediction of building energy   consumption by using artificial neural networks.                                                                                                                             | Advances in Engineering   Software.                          | 2009 | 358               |
| Mocanu,   E., Nguyen, P. H., Gibescu, M., y Kling, W. L.      | Deep learning for estimating   building energy consumption.                                                                                                                                                  | Sustainable Energy, Grids and   Networks.                    | 2016 | 342               |
| Edwards,   R. E., New, J., y Parker, L. E.                    | Predicting future hourly   residential electrical consumption: A machine learning case study.                                                                                                                | Energy and Buildings.                                        | 2012 | 275               |
| Ahmad,   M. W., Mourshed, M., y Rezgui, Y.                    | Trees vs Neurons: Comparison   between random forest and ANN for high-resolution prediction of building   energy consumption.                                                                                | Energy and Buildings                                         | 2017 | 263               |
| Tüfekci,   P.                                                 | Prediction of full load   electrical power output of a base load operated combined cycle power plant   using machine learning methods.                                                                       | International Journal of   Electrical Power & Energy Systems | 2014 | 241               |
| Candanedo,   L. M., Feldheim, V., y Deramaix, D.              | Bélgica                                                                                                                                                                                                      | Energy and buildings                                         | 2017 | 2                 |




# 1  Introduccion

El análisis exploratorio de datos es una etapa fundamental en cualquier proyecto de Inteligencia Artificial y Ciencia de Datos, puesto que permite entender las características y patrones en los datos, así como identificar relaciones entre variables.

A continuación se presenta un estudio basado en datos de consumo de energía eléctrica horaria registrados en Cali, Colombia, junto con las temperaturas correspondientes a cada hora. El objetivo principal es explorar la relación entre estas dos variables y, posteriormente, aplicar diferentes modelos para realizar el pronósticos de corto plazo del demanda de energía.

# 2  Contexto

La demanda de energía eléctrica en una ciudad se ve inlfuenciada por muchos factores, como la temperatura, factores sociales, cronograma de produccion de grandes clientes, entre otros.

En regiones con climas variables, como es el caso de muchas ciudades Colombianas, la demanda es afectada por el uso intensivo de sistemas de refrigeración en épocas calurosas.

Los datos utilizados en este análisis incluyen mediciones horarias del consumo de energía y la temperatura registrada en la misma franja horaria, tipos de dia, mes y año. Lo que permite explorar la correlación entre estas variables.

Adicionalmente, se busca evaluar el desempeño de diferentes modelos como regresiones lineales, redes neuronales y modelos basados en árboles de decisión, para predecir el consumo de energía en un horizonte de corto plazo.

## 2.1  Descripción de los campos

- FECHA: Fecha en formato ISO de la lectura del consumo
- TEMPERATURA: Temperatura en grados Centigrados al momento de la toma del dato
- CONSUMO: Variable objetvio medido en MVA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import holidays
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

df = pd.read_csv('Data/datasetEmcali.csv')



In [ ]:
print("primeras filas \n")
print(df.head())


print("se describen las columnas \n")
print(df.info())

print("dimensiones \n")
print(df.shape)

# 3  Análisis exploratorio


## 3.1  Análisis de estructura


Se realiza un analisis de la forma del dataset, se relacionan tipos de datos, estadisticas descriptivas y valores nulos.

Se adicionan columnas, basadas en la fecha, con la hipotesis que esto permitira al modelo entender e interpretar mejor los datos.

Tratamiento de dataset Humedad

In [ ]:
#Defino el formato de la fecha
df["Date"] = pd.to_datetime(df["FECHA"], format="mixed", errors="coerce")

# Cargar datasets de humedad
humedad_files = glob.glob("Data/reporte_sisaire-humedad-*.csv")
df_humedad_list = [pd.read_csv(file) for file in humedad_files]
df_humedad = pd.concat(df_humedad_list)

# Convertir fechas en el dataset de humedad
df_humedad["Fecha inicial"] = pd.to_datetime(df_humedad["Fecha inicial"], format="%Y-%m-%d %H:%M")

# Promediar la humedad por hora
df_humedad_avg = df_humedad.groupby("Fecha inicial")["HAire2"].mean().reset_index()
df_humedad_avg.rename(columns={"Fecha inicial": "Date", "HAire2": "Humidity"}, inplace=True)

df_humedad_avg.head()

Tratamiento de dataset Temperatura

In [ ]:
# Cargar datasets de velocidad del viento
viento_files = glob.glob("Data/reporte_sisaire-vViento-*.csv")
df_viento_list = [pd.read_csv(file) for file in viento_files]
df_viento = pd.concat(df_viento_list)

# Convertir fechas en el dataset de viento
df_viento["Fecha inicial"] = pd.to_datetime(df_viento["Fecha inicial"], format="%Y-%m-%d %H:%M")

# Promediar la velocidad del viento por hora
df_viento_avg = df_viento.groupby("Fecha inicial")["VViento"].mean().reset_index()
df_viento_avg.rename(columns={"Fecha inicial": "Date", "VViento": "Wind"}, inplace=True)

df_viento_avg.head()

Unión de datos de dos DataFrames, df_humedad_avg y df_viento_avg, con el DataFrame principal df

In [ ]:
df_humedad_avg["Date"] = pd.to_datetime(df_humedad_avg["Date"])
df_viento_avg["Date"] = pd.to_datetime(df_viento_avg["Date"])

df = df.merge(df_humedad_avg, on="Date", how="left", suffixes=("","_extra"))
df = df.merge(df_viento_avg, on="Date", how="left" , suffixes=("","_extra"))

Se realiza la Conversion de Columnas a Valores Numericos y se eliminnan las filas que tienen valores nulos en 'FECHA' y 'CONSUMO', 'TEMPERATURA', 'HUMEDAD', 'VIENTO'

In [ ]:
df['Consumption'] = pd.to_numeric(df['CONSUMO'], errors='coerce')

df['Temperature'] = pd.to_numeric(df['TEMPERATURA'], errors='coerce')

df['Humidity'] = pd.to_numeric(df['Humidity'], errors='coerce')

df['Wind'] = pd.to_numeric(df['Wind'], errors='coerce')

# Elimino las filas que tienen valores nulos

df = df.dropna(subset=['Date', 'Consumption', 'Temperature', 'Humidity', 'Wind'])

Se Verifica si la fecha es un dia festivo en Colombia

In [ ]:
def es_festivo_colombia(fecha):
    """Verifica si una fecha es festivo en Colombia."""
    co_holidays = holidays.CO(years=fecha.year)
    return fecha in co_holidays
df['Is_holiday'] = df['Date'].apply(es_festivo_colombia)

Se decompone la fecha, se crean nuevas columnas basados en sus componentes, Esto puede ser útil para análisis de series temporales o para extraer información adicional de las fechas.

In [ ]:
# se descompone la fecha
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Dayofweek'] = df['Date'].dt.dayofweek
df['Dayofyear'] = df['Date'].dt.dayofyear
df['Weekofyear'] = df['Date'].dt.isocalendar().week
df['Is_month_end'] = df['Date'].dt.is_month_end
df['Is_month_start'] = df['Date'].dt.is_month_start
df['Is_quarter_end'] = df['Date'].dt.is_quarter_end
df['Is_quarter_start'] = df['Date'].dt.is_quarter_start
df['Is_year_end'] = df['Date'].dt.is_year_end
df['Is_year_start'] = df['Date'].dt.is_year_start

Se eliminan columnas innecesarias

In [ ]:
# Eliminar la columna 'FECHA', ya que no es necesaria
df = df.drop('Date', axis=1)
df = df.drop('FECHA', axis=1)
df = df.drop('TEMPERATURA', axis=1)
df = df.drop('CONSUMO', axis=1)

In [ ]:
# Convert categorical columns to categorical
categorical_cols = ['Year', 'Month', 'Day', 'Hour', 'Dayofweek', 'Dayofyear', 'Weekofyear']
for col in categorical_cols:
    df[col] = df[col].astype('category')

print("Forma de lo datos:", df.shape)
print("\nTipo de datos por columnas:\n\n", df.dtypes)
print("\nEstadisticas descriptivas:\n\n", df.describe())
print("\nValores Faltantes:\n\n", df.isnull().sum())
print("\nValores Faltantes(%):\n\n", (df.isnull().sum() / len(df)) * 100)

se notan valores nulos

## 3.2  Análisis de variables numericas y continuas


In [ ]:

# Se define las columnas numericas a analizar
numerical_columns = ['Consumption', 'Temperature', 'Humidity', 'Wind']

# PRIMERO => Matriz de Correlacion
# Se calcula la matriz de correlacion entre ellas
correlation_matrix = df[numerical_columns].corr()

# Configuracion del grafico para la matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(
    correlation_matrix,
    annot=True, #Muestra los valores de correlacion
    cmap='RdYlGn', #Paleta de Colores(rojo-amarrillo-verde)
    fmt='0.4f', #Formato de 4 decimales
    vmin=-1, vmax=1, #Rango de valores para la escala de colores
)
plt.title('Correlation Matrix of Numerical Columns')
plt.show()
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# SEGUNDO => Analisis de Distribucion y Relacion con Consumo
# Se configura el diselo de subplots
num_cols = 2 # Numero de columnas por fila
num_rows = (len(numerical_columns)) # Una fila por variable

# Crear la figura y los subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))
axes = axes.flatten() # Convertir la matriz de subplots en una lista plana

# Iteracion sobre cada columna numérica
for i, col in enumerate(numerical_columns):
    # Histograma: Distribucion de la variable
    sns.histplot(df[col], bins=20, kde=True, ax=axes[i * 2], color='royalblue') # KDE para visualizar la distribución
    axes[i * 2].set_title(f'Distribution of {col}', fontsize=14)
    axes[i * 2].set_xlabel(col)
    axes[i * 2].set_ylabel('Frequency')
    
    # Grafico de disperson: Relacion con CONSUMO
    sns.regplot(
        x=col,
        y='Consumption',
        data=df,
        ax=axes[i * 2 + 1],
        scatter_kws={'alpha': 0.5}, # Transparencia para mejorar la vista
        line_kws={'color': 'red'}, # Color de la línea de tendencia
    )
    axes[i * 2 + 1].set_title(f'CONSUMO vs. {col}', fontsize=14)
    axes[i * 2 + 1].set_xlabel(col) 
    axes[i * 2 + 1].set_ylabel('CONSUMO') 
    
    # Calcular y mostrar la correlacion
    correlation = df[col]. corr(df['Consumption'])
    print(f"Correlation between {col} and CONSUMO: {correlation:.4f}")

# Eliminar subplots no utilizados
for j in range(len(numerical_columns) * 2, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()




## 3.3  Análisis de variables Categoricas


Las columna fechas contiene información temporal (día, mes, año, hora, etc.), sin embargo, para que los diferentes modelos pueda aprovechar la información de la variable fecha, es necesario transformarla en características categóricas.
A continuación, se trata la columna fecha:

### Crear columna con **Dias festivos** en **Colombia**

In [ ]:
num_cols = 4
num_rows = (len(categorical_cols) * 2 + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))
axes = axes.flatten()

for i, col in enumerate(categorical_cols):
    sns.boxplot(x=col, y='Consumption', data=df, ax=axes[i])
    axes[i].set_title(f'Consumption vs. {col}')

    # Remove any unused subplots
for j in range(len(categorical_cols), num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

- En la variable "Year" se puede evidenciar que los años 2020 y 2021 presentaron fenomenos sociales que impactaron directamente el consumo de eneriga en la ciudad de Cali. No existe en el dataset, una variable que permita modelar las restricciones a la movilidad en la pandemia y el paro nacional. *Se concidera prudente, sacar del estudio estos dos años.*
- Se puede observar de "Dayofweek" que el consumo en los dias 1 a 4 tiene un comportamiento similar, en cambio los días 0, 5 y 6 se diferencian de los otros dias de la semana.



### Analisis de outliers y datos Atipicos


#### Eliminar datos atipicos

Como se menciono antes, se procede a eliminar los datos de los años 2020 y 2021.

In [ ]:
df = df.drop(df[(df['Year'] == 2020) | (df['Year'] == 2021)].index)
df = df.reset_index(drop=True)

num_cols = 4
num_rows = (len(categorical_cols) * 2 + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))
axes = axes.flatten()

for i, col in enumerate(categorical_cols):
    sns.countplot(x=col, data=df, ax=axes[i * 2])
    axes[i * 2].set_title(f'Frequency Distribution of {col}')
    sns.boxplot(x=col, y='Consumption', data=df, ax=axes[i * 2 + 1])
    axes[i * 2 + 1].set_title(f'Consumption vs. {col}')

for j in range(len(categorical_cols) * 2, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()  # Adjust subplot parameters to give specified padding.
plt.show()

#### Analisis de Outliers

In [ ]:
# Columnas numéricas
numerical_cols = ['Consumption', 'Temperature', 'Humidity', 'Wind']
# Box plots to visualize outliers
plt.figure(figsize=(12, 6))
for i, col in enumerate(numerical_cols):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(y=df[col])
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.show()

# Descriptive statistics and IQR outlier detection
for col in numerical_cols:
    print(f"\nAnalysis for numerical variable: {col}")
    print(df[col].describe())

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Number of outliers: {len(outliers)}")
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=col, y='Consumption', data=df)
    plt.title(f'Consumption vs. {col} with Outliers')

    # Highlight outliers on the scatter plot
    if not outliers.empty:
        plt.scatter(outliers[col], outliers['Consumption'], color='red', label='Outliers')
        plt.legend()

    plt.show()

**Conclusiones Outliers**
- Se pudo observar un dato en la variable temperatura que se identificó como Outlier, sin embargo, este pudo obedecer a una temporada sin lluvia, en donde la temperatura se elevó a valores superiores.
- Para la variable Consumo, de acuerdo a lo manifestado por los expertos del negocio, los valores bajos se puden considedar como outliers, causados por apagones o fallas en la red de distribucion de energia.

In [ ]:
df.isnull().sum()

# 4  Imputación de datos


In [ ]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
df_cleaned = df.copy()

# Crea series desplazadas para el valor anterior y siguiente
temperatura_anterior = df_cleaned['Temperature'].shift(1)
temperatura_siguiente = df_cleaned['Temperature'].shift(-1)

# Calcula el promedio del valor anterior y siguiente
promedio_anterior_siguiente = (temperatura_anterior + temperatura_siguiente) / 2

# Imputa los valores faltantes con el promedio
df_cleaned['Temperature'] = df_cleaned['Temperature'].fillna(promedio_anterior_siguiente)

# Manejar los valores NaN restantes al inicio o al final del DataFrame
df_cleaned['Temperature'] = df_cleaned['Temperature'].fillna(df_cleaned['Temperature'].median()) # Corregido

# Reassign the cleaned DataFrame to df
df = df_cleaned

print(df.head())
print(df.info())
print(df.isnull().sum())

Defino funcion enfocada para la ejecucion de los Pipelines

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class TemperatureImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # No es necesario entrenar el modelo, así que simplemente retorna self
        return self
    
    def transform(self, X):
        # Crear una copia del DataFrame para evitar la advertencia SettingWithCopyWarning
        df_cleaned = X.copy()

        # Crea series desplazadas para el valor anterior y siguiente
        temperatura_anterior = df_cleaned['Temperature'].shift(1)
        temperatura_siguiente = df_cleaned['Temperature'].shift(-1)

        # Calcula el promedio del valor anterior y siguiente
        promedio_anterior_siguiente = (temperatura_anterior + temperatura_siguiente) / 2

        # Imputa los valores faltantes con el promedio
        df_cleaned['Temperature'] = df_cleaned['Temperature'].fillna(promedio_anterior_siguiente)

        # Manejar los valores NaN restantes al inicio o al final del DataFrame
        df_cleaned['Temperature'] = df_cleaned['Temperature'].fillna(df_cleaned['Temperature'].median())

        return df_cleaned


- En el dataset estudiado no hay presencia de datos faltantes, se contempla el proceso de imputacion para la variable **Temperatura** debido a la posibilidad de la llegada de nuevos datos, a travez del cual se puede imputar los valores nulos.
- Para imputar se tiene en cuenta la medida anterior y posterior con referencia al dato nulo, se realiza un promedio de estas dos medidas, teniendo en cuenta el vector.

## 4.1  Division de datasets


In [ ]:
y = df['Consumption']
X = df.drop(['Consumption'], axis=1)

print("primeras filas \n")
print(X.head())

print("se describen las columnas \n")
print(X.info())

print("dimensiones \n")
print(X.shape)
print("dimensiones \n")
print(y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 4.2  Estandarización y Escalamiento


Teniendo en cuenta que solo la variable temperatura es numerica y continua no se requiere Escalar o normalizar.

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 5  Feature Engineering


## 5.1  Codificar Variables Categoricas


Como las variables categoricas del dataset no tienen una cardinalidad, se usa la codificación One Hot Encoding, para su aplicación en el modelo.

In [ ]:
# Use pd.get_dummies with the 'columns' parameter to specify which columns to dummify
X_train = pd.get_dummies(X_train, columns=categorical_cols)
X_test = pd.get_dummies(X_test, columns=categorical_cols)
#print(X_train.head())

# Check if the number of columns in X_train and X_test are equal
print(f"Number of columns in X_train: {X_train.shape[1]} \n")
print(f"Number of columns in X_test: {X_test.shape[1]}")


# 6 Regularización

In [ ]:
from sklearn.linear_model import Lasso

# Initialize the Lasso model with a regularization parameter alpha
lasso_model = Lasso(alpha=0.1)

# Train the model
lasso_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate the model
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)

# Print the evaluation metrics
print(f"Lasso Model - Mean Squared Error (MSE): {mse_lasso}")
print(f"Lasso Model - R-squared (R2): {r2_lasso}")
print(f"Lasso Model - Mean Absolute Error (MAE): {mae_lasso}")
print(f"Lasso Model - Root Mean Squared Error (RMSE): {rmse_lasso}")

# Print the coefficients
print('Lasso Coefficients:', lasso_model.coef_)


In [ ]:
# Contar cuántos coeficientes no son cero
num_nonzero_coefficients = np.sum(lasso_model.coef_ != 0)

print(f"Número de coeficientes no cero después de aplicar Lasso: {num_nonzero_coefficients}")

- Podemos observar que LASSO redujo notablemente los coeficientes de 252 a 78 Se puede observar que Lasso bajo la dimensionalidad de los datos, con 178 coeficientes iguales a cero.
Lasso es útil para la selección de características.

- Adicinalmente el $R^2$ de Lasso es menor que el de lo demás modelos, pero Lasso solo uso solo 78 coeficientes mientras que los otros dos usaron 252 coeficientes. El modelo Lasso es más factible porque las dimensiones son reducidas.

-Por otro lado, se puede afirmar que el modelo de Ridge es mucho más costoso en términos operativos, y aunque los valores de los errores son muy similares , El método Lasso es mejor para la selección de características porque puede reducir los coeficientes sin importancia a cero, eliminando de manera efectiva esas características. Mientras que la regresión Ridge reduce todos los coeficientes pero no elimina ninguna característica.

In [ ]:
from sklearn.linear_model import Ridge

# Initialize the Ridge model with a regularization parameter alpha
ridge_model = Ridge(alpha=0.1)

# Train the model
ridge_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate the model
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)

# Print the evaluation metrics
print(f"Ridge Model - Mean Squared Error (MSE): {mse_ridge}")
print(f"Ridge Model - R-squared (R2): {r2_ridge}")
print(f"Ridge Model - Mean Absolute Error (MAE): {mae_ridge}")
print(f"Ridge Model - Root Mean Squared Error (RMSE): {rmse_ridge}")

# Print the coefficients
print('Ridge Coefficients:', ridge_model.coef_)
# Count the number of coefficients that are not equal to 0
num_nonzero_coefficients_ridge = np.sum(ridge_model.coef_ != 0)

print(f"Number of non-zero coefficients in Ridge model: {num_nonzero_coefficients_ridge}")

# 7 Data Pipeline


### Pipeline para probar el modelo de regresion random forest

In [ ]:

#division del dataset para entrenamiento y prueba usando el pipeline
X_train_to_pipe, X_test_to_pipe, y_train_to_pipe, y_test_to_pipe = train_test_split(X, y, test_size=0.2, random_state=42)

#codifico las variables categoricas
X_train_to_pipe = pd.get_dummies(X_train_to_pipe, columns=categorical_cols)
X_test_to_pipe = pd.get_dummies(X_test_to_pipe, columns=categorical_cols)


# Pipeline para probar el modelo de regresion random forest
pipe = Pipeline([
  ('Imputacion temperatura', TemperatureImputer()),
  ('Estandarizacion', StandardScaler()),
  ('Model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),)
],verbose = True)


# Entrenamiento del pipeline
pipe.fit(X_train_to_pipe, y_train_to_pipe)


# Evaluar el pipeline 
print("Accuracy:",pipe.score(X_test_to_pipe, y_test_to_pipe))



### Pipeline para la prueba de un modelo de regresión lineal

In [ ]:
# Crear el pipeline con regresión lineal
pipe = Pipeline([
  ('Imputacion_temperatura', TemperatureImputer()),  
  ('Estandarizacion', StandardScaler()), 
  ('model', LinearRegression())  
], verbose=True)

# Entrenamiento del pipeline
pipe.fit(X_train_to_pipe, y_train_to_pipe)

# Evaluar el pipeline
print("Accuracy:", pipe.score(X_test_to_pipe, y_test_to_pipe))

# 7  Modelamiento

### Modelo hibrido XGBoost + DNN

In [ ]:
import xgboost as xgb
from tensorflow import keras

# 1. Define and train the DNN model
dnn_model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)  # Output layer for regression
])

dnn_model.compile(optimizer='adam', loss='mse')
dnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
 
# 2. Use the DNN model to generate predictions for the training and test sets
dnn_train_predictions = dnn_model.predict(X_train)
dnn_test_predictions = dnn_model.predict(X_test)

# 3. Combine the original features with the DNN predictions
X_train_hybrid = np.hstack((X_train, dnn_train_predictions))
X_test_hybrid = np.hstack((X_test, dnn_test_predictions))

# 4. Train the XGBoost model using the hybrid features
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=10, random_state=42)
xgb_model.fit(X_train_hybrid, y_train)

# 5. Make predictions with the XGBoost model
y_pred_hybrid = xgb_model.predict(X_test_hybrid)

# 6. Evaluate the hybrid model
mse_hybrid = mean_squared_error(y_test, y_pred_hybrid)
r2_hybrid = r2_score(y_test, y_pred_hybrid)
mae_hybrid = mean_absolute_error(y_test, y_pred_hybrid)
rmse_hybrid = np.sqrt(mse_hybrid)

print(f"Hybrid Model - Mean Squared Error (MSE): {mse_hybrid}")
print(f"Hybrid Model - R-squared (R2): {r2_hybrid}")
print(f"Hybrid Model - Mean Absolute Error (MAE): {mae_hybrid}")
print(f"Hybrid Model - Root Mean Squared Error (RMSE): {rmse_hybrid}")

### Modelo RegresionLineal

In [ ]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Calculate MAE and MedAE
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print('Intercepto: ', model.intercept_)
print('Coeficientes: ', model.coef_)

In [ ]:
# Count the number of coefficients greater than 0
num_positive_coefficients = np.sum(model.coef_ != 0)

print(f"Number of coefficients greater than 0: {num_positive_coefficients}")

### Modelo Randomforest

In [ ]:
# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Print the evaluation metrics
print(f"Random Forest - Mean Squared Error (MSE): {mse_rf}")
print(f"Random Forest - R-squared (R2): {r2_rf}")

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_rf)

# Calculate the Median Absolute Error (MedAE)
medae = median_absolute_error(y_test, y_pred_rf)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))

# Print the calculated MAE, MedAE, and RMSE values.
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

### Modelo de Redes Neuronales

In [ ]:
# 1. Definir el modelo de la red neuronal
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)  # Capa de salida para regresión
])

# 2. Compilar el modelo
model.compile(optimizer='adam', loss='mse')  # Usamos 'mse' (error cuadrático medio) para regresión

# 3. Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# 4. Realizar predicciones
y_pred = model.predict(X_test)

# 5. Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

# Opcional: Graficar la pérdida durante el entrenamiento
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()